In [1]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import pandas as pd
import sys
from datetime import datetime
from IPython.display import display
import yaml
from pathlib import Path

pd.options.display.max_colwidth = 300
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    print('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
except ImportError as err:
    print('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err

source_code_version_id = "f3db7e42d52ed8844421d3124358e0717f0b9a2d"
notebook.setup("INFO", source_code_version_id)

import notebooks.standards_library.standard_annotation as sta

2025-05-06 19:06:12 INFO     Running on git commit: b'f3db7e42d52ed8844421d3124358e0717f0b9a2d' from 2025-05-06 11:32:18 -0700 -- 34 minutes ago


2025-05-06 19:06:13 INFO     NERSC=True


# Read config file

In [2]:
config_path = "/global/homes/b/bkieft/metatlas/notebooks/standards_library/ref_std_annotation.yaml"
with open(config_path, "r") as config_file:
    config = yaml.safe_load(config_file)

# Extract EIC and Spectra information from files in the run table

In [3]:
if config["full_data_from_cache"] is False:
    lcmsruns_table_with_adducts = sta.build_standard_lcmsrun_table(config)
    eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, mols_images = sta.extract_data(lcmsruns_table_with_adducts,config,method="find_peaks")
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="full", data=(eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, mols_images))
elif config["full_data_from_cache"] is True:
    eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, mols_images = sta.handle_data(mode="load", config=config, file_suffix="full")

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 

[19:07:40] ERROR: 



 Extracting data from lcmsruns:   0%|          | 0/120 [00:00<?, ?compound group/s]

Saving data to: /global/homes/b/bkieft/metatlas_junkdrawer/example_data/schellermetasci/cache/20250506190611_ref_stds_full.pkl


# Create interactive plot to choose adduct rt peaks for each standard compound

In [99]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import base64
import math
from typing import Dict, List, Tuple, Union, Any, Optional, Set


def create_interactive_plots(
    processed_data: List[Dict[str, Any]],
    runnum_to_structure_image_grid: Dict[int, str],
    selected_good_adducts: Dict[str, List[str]],
    ambiguous_adducts: Dict[str, str],
    top_adducts: Dict[str, List[str]]
) -> None:
    """
    Create interactive plots for visualizing and annotating processed data.

    Args:
        processed_data (List[Dict[str, Any]]): A list of dictionaries containing processed data for each compound group.
            Each dictionary should include keys such as 'eics', 'rt_peaks', 'top_spectra', 'adduct_color', 'group_id',
            'unique_id', 'group_file', and 'compound_name'.
        runnum_to_structure_image_grid (Dict[int, str]): A dictionary mapping run numbers to base64-encoded molecular structure images.
        selected_good_adducts (Dict[str, List[str]]): A dictionary mapping unique compound IDs to lists of selected
            adduct-peak combinations in the format "adduct||peak_index".
        ambiguous_adducts (Dict[str, str]): A dictionary mapping unique compound IDs to ambiguous annotations.
        top_adducts (Dict[str, List[str]]): A dictionary mapping unique compound IDs to lists of top adducts.

    Returns:
        None
    """
    
    # Create a state object to track current values
    class PlotState:
        def __init__(self):
            self.current_index = 0
            self.current_unique_id = processed_data[0]['unique_id'] if processed_data else None
            # Add debug counters
            self.update_count = 0
    
    state = PlotState()
    
    # Create persistent widgets (those that don't need to be recreated)
    image_toggle = widgets.ToggleButton(
        value=False,
        description='Show Structures',
        tooltip='Toggle to show/hide the compound structure image',
        layout=widgets.Layout(width='150px', margin='5px 0 0 0')
    )
    yaxis_toggle = widgets.ToggleButton(
        value=False,
        description='Shared Y-Axis',
        tooltip='Toggle between unique and shared y-axes for non-log EIC plots',
        layout=widgets.Layout(width='150px', margin='30px 0 0 0')
    )
    next_button = widgets.Button(description="Next Group")
    previous_button = widgets.Button(description="Previous Group")
    progress_label = widgets.Label(value=f"1/{len(processed_data)} Groups Completed")
    navigate_textbox = widgets.Text(
        placeholder='Index...',
        description='Go to:',
        layout=widgets.Layout(width='50px')
    )
    navigate_button = widgets.Button(
        description="Go",
        layout=widgets.Layout(width='50px')
    )
    compound_image_widget = widgets.Image(
        format='png',
        layout=widgets.Layout(
            width='400px',
            height='400px',
            margin='0 0 0 50px',
            display='none'  # Initially hidden
        )
    )
    compound_image_label = widgets.Label(
        value="Compound Structures",
        layout=widgets.Layout(margin='0 0 10px 0')
    )
    completion_label = widgets.Label(value="", layout=widgets.Layout(margin="0 0 0 0"))
    output_container = widgets.Output()
    
    # Main output area that will be refreshed
    main_output = widgets.Output()
    
    # Define event handlers for persistent widgets
    def on_image_toggle_change(change):
        if image_toggle.value:
            image_toggle.description = 'Hide Structures'
            compound_image_widget.layout.display = 'block'
        else:
            image_toggle.description = 'Show Structures'
            compound_image_widget.layout.display = 'none'
    
    def update_progress_text():
        progress_label.value = f"{state.current_index + 1}/{len(processed_data)} Groups Completed"
    
    def on_toggle_change(change):
        yaxis_toggle.description = 'Shared Y-Axis' if not yaxis_toggle.value else 'Unique Y-Axis'
        update_display()
    
    def next_group(b):
        if state.current_index < len(processed_data) - 1:
            state.current_index += 1
            state.current_unique_id = processed_data[state.current_index]['unique_id']
            image_toggle.value = False
            update_display()
            completion_label.value = ""
        else:
            completion_label.value = "Analysis completed!"
    
    def previous_group(b):
        if state.current_index > 0:
            state.current_index -= 1
            state.current_unique_id = processed_data[state.current_index]['unique_id']
            image_toggle.value = False
            update_display()
            completion_label.value = ""
        else:
            output_container.clear_output(wait=True)
            with output_container:
                print("Already at the first group.")
    
    def navigate_to_group(b):
        try:
            target_index = int(navigate_textbox.value) - 1
            if 0 <= target_index < len(processed_data):
                state.current_index = target_index
                state.current_unique_id = processed_data[state.current_index]['unique_id']
                update_display()
            else:
                output_container.clear_output(wait=True)
                with output_container:
                    print(f"Invalid index. Please enter a number between 1 and {len(processed_data)}.")
        except ValueError:
            output_container.clear_output(wait=True)
            with output_container:
                print("Invalid input. Please enter a valid integer.")
    
    # Bind event handlers
    image_toggle.observe(on_image_toggle_change, names='value')
    yaxis_toggle.observe(on_toggle_change, names='value')
    next_button.on_click(next_group)
    previous_button.on_click(previous_group)
    navigate_button.on_click(navigate_to_group)
    
    # Helper function to directly update dictionaries when checkboxes change
    def create_checkbox_handlers(unique_id, all_adducts_checkboxes, adduct_peak_combinations, ambiguous_checkbox, top_adducts_checkboxes):
        """Creates handlers that explicitly use the provided unique_id"""
        
        def on_all_adducts_change(change):
            
            if ambiguous_checkbox.value:
                # Mark as ambiguous
                ambiguous_adducts[unique_id] = unique_id
                # Remove from other dictionaries
                if unique_id in selected_good_adducts:
                    del selected_good_adducts[unique_id]
                if unique_id in top_adducts:
                    del top_adducts[unique_id]
                
            else:
                # Get all selected adduct combinations
                selected_combos = []
                checkbox_dict = {c.description: c for c in all_adducts_checkboxes[:-1]}  # Exclude ambiguous checkbox
                
                for combo in adduct_peak_combinations:
                    if combo['description'] in checkbox_dict and checkbox_dict[combo['description']].value:
                        selected_combos.append(f"{combo['adduct']}||{combo['peak_index']}")
                
                # Update dictionary only if selections were made
                if selected_combos:
                    selected_good_adducts[unique_id] = selected_combos
                elif unique_id in selected_good_adducts:
                    del selected_good_adducts[unique_id]
                
                # Remove from ambiguous
                if unique_id in ambiguous_adducts:
                    del ambiguous_adducts[unique_id]
        
        def on_top_adducts_change(change):
            selected = []
            for checkbox in top_adducts_checkboxes.children:
                if checkbox.value:
                    selected.append(checkbox.description)
            
            if selected:
                top_adducts[unique_id] = selected
            elif unique_id in top_adducts:
                del top_adducts[unique_id]
        
        return on_all_adducts_change, on_top_adducts_change

    # Layout Definitions
    def create_layout(all_adducts_checkboxes, top_adducts_checkboxes):
        checkbox_layout = widgets.VBox(
            children=[
                widgets.Label(value="Select all good adducts:"),
                *all_adducts_checkboxes
            ],
            layout=widgets.Layout(
                border='1px solid black',
                padding='10px',
                margin='10px',
                width='300px',
                align_items='flex-start'  # Align items to the start (left)
            )
        )
        top_adducts_checkboxes_layout = widgets.VBox(
            children=[
                widgets.Label(value="Select best adduct(s) - default intensity:"),
                top_adducts_checkboxes
            ],
            layout=widgets.Layout(
                border='1px solid black',
                padding='10px',
                margin='10px',
                width='300px',
                align_items='flex-start'
            )
        )
        go_to_label = widgets.Label(value="Go To:")
        go_to_layout = widgets.HBox(
            [go_to_label, navigate_textbox, navigate_button],
            layout=widgets.Layout(
                justify_content='flex-start',  # Align to the far left
                spacing='5px',
                margin='30px 0 0 0'  # Add space above the widget
            )
        )
        navigate_textbox.description = ""
        navigate_textbox.layout = widgets.Layout(width='150px')  # Decrease the size of the search box
    
        compound_image_widget.layout.display = 'none'
        image_toggle.layout.margin = '0 0 0 50px'
    
        navigation_buttons_layout = widgets.HBox(
            [
                widgets.VBox([next_button, previous_button]),  # Stack Previous and Next buttons vertically
                image_toggle  # Place the Image Toggle button to the right
            ],
            layout=widgets.Layout(
                justify_content='flex-start',  # Align items to the left
                spacing='10px',  # Add spacing between elements
                margin='0 0 0 0'  # No margin for the navigation buttons
            )
        )
        button_layout = widgets.VBox(
            [navigation_buttons_layout, progress_label, go_to_layout, yaxis_toggle],
            layout=widgets.Layout(
                align_items='flex-start',
                spacing='5px'
            )
        )
        compound_image_container = widgets.VBox(
            [compound_image_label, compound_image_widget],
            layout=widgets.Layout(
                align_items='center',  # Center-align the content
                padding='10px',  # Add padding around the container
                border='1px solid lightgray',  # Optional: Add a border for better visibility
                width='500px'  # Ensure the container is slightly wider than the image
            )
        )
        top_layout = widgets.HBox(
            [checkbox_layout, top_adducts_checkboxes_layout, button_layout, compound_image_container],
            layout=widgets.Layout(
                align_items='flex-start',
                justify_content='flex-start',
                spacing='10px'
            )
        )
        final_layout = widgets.VBox(
            [completion_label, top_layout],
            layout=widgets.Layout(
                align_items='flex-start',  # Center-align the content
                padding='0px',  # Add padding around the container
            )
        )
        return final_layout
    
    # Main display update function that recreates the plots and widgets
    def update_display():
        state.update_count += 1
        main_output.clear_output(wait=True)
        
        # Get current data
        data = processed_data[state.current_index]
        unique_id = data['unique_id']
        state.current_unique_id = unique_id

        # Get data from current group
        eics = data['eics']
        top_spectra = data['top_spectra']
        rt_peaks = data['rt_peaks']
        adduct_color = data['adduct_color']
        group_id = data['group_id']
        unique_id = data['unique_id']
        group_run_number = data['group_run_number']

        # Extract adduct-peak combinations from rt_peaks and top_spectra
        adduct_peak_combinations = []
        if isinstance(rt_peaks, pd.DataFrame) and not rt_peaks.empty:
            # Create a mapping from adducts to peak indices and intensities
            adduct_to_peaks = {}
            for _, peak_row in rt_peaks.iterrows():
                adduct = peak_row['adduct'] if 'adduct' in peak_row else None
                if adduct:
                    if adduct not in adduct_to_peaks:
                        adduct_to_peaks[adduct] = []
                    adduct_to_peaks[adduct].append({
                        'peak_index': peak_row['peak_index'],
                        'intensity': peak_row['intensity']
                    })

        # Create unique identifiers for each adduct-peak combination
        for adduct, peaks in adduct_to_peaks.items():
            max_intensity = max(peak['intensity'] for peak in peaks)
            for peak in peaks:
                # Check if there is an MS2 spectrum for this adduct+peak_index
                if top_spectra.empty:
                    has_ms2 = False
                else:
                    has_ms2 = not top_spectra[
                        (top_spectra['adduct'] == adduct) & 
                        (top_spectra['peak_index'] == peak['peak_index'])
                    ].empty

                # Add a star to the description if MS2 spectrum exists
                description = f"{adduct} ({peak['peak_index']}){' *' if has_ms2 else ''}"
                adduct_peak_combinations.append({
                    'adduct': adduct,
                    'peak_index': peak['peak_index'],
                    'description': description,
                    'max_intensity': max_intensity
                })

        # Sort adduct_peak_combinations by max_intensity in descending order
        adduct_peak_combinations.sort(key=lambda x: x['max_intensity'], reverse=True)

        # Create the summary EIC plot data
        group_run_eics = [
            eic for pdata in processed_data if pdata['group_run_number'] == group_run_number
            for eic in pdata['eics'].values()
        ]
        summary_traces = []
        summary_xmin_list = []
        summary_xmax_list = []
        for eic in group_run_eics:
            # Loop through each row in the eic DataFrame
            for _, eic_row in eic.iterrows():
                # Filter data where intensity is above 1e5
                valid_indices = eic_row['i'] > 1e5
                filtered_rt = eic_row['rt'][valid_indices]
                filtered_i = eic_row['i'][valid_indices]

                if len(filtered_rt) > 0:  # Ensure there are valid points
                    # Sort retention times
                    rt_sort = np.argsort(filtered_rt)
                    adduct = sta.get_adduct(eic_row['label'])  # Extract adduct from the label
                    color = adduct_color.get(adduct, 'gray')  # Default to gray if adduct color is missing
                    label = eic_row['label']

                    # Update x_min and x_max based on filtered data
                    summary_xmin_list.append(filtered_rt.min())
                    summary_xmax_list.append(filtered_rt.max())

                    # Add a trace for the current adduct
                    summary_traces.append(
                        go.Scatter(
                            x=filtered_rt[rt_sort],
                            y=filtered_i[rt_sort],
                            mode='lines',
                            name=f"{label}",
                            line=dict(color=color),
                            showlegend=False
                        )
                    )
        x_min = min(summary_xmin_list) if summary_xmin_list else None
        x_max = max(summary_xmax_list) if summary_xmax_list else None

        # Create the figure with subplots
        num_spectra = len(top_spectra)
        if num_spectra == 0:
            num_spectra = 1  # Ensure at least one row for empty top_spectra
        num_columns = 4
        num_spectra_rows = math.ceil(num_spectra / num_columns)

        # Adjust subplot titles and specifications
        subplot_titles = [
            "Sample",
            "Blank",
            "EIC Summary",
            "Sample (Log)",
            "Blank (Log)",
            *(f"" if top_spectra.empty else f"{row['adduct']} @ {round(row['rt'], 2)} mins" for _, row in top_spectra.iterrows())
        ]

        specs = [
            [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter", "rowspan": 2, "colspan": 2}, None],
            [{"type": "scatter"}, {"type": "scatter"}, None, None],
            *[[{"type": "scatter"} for _ in range(4)] for _ in range(num_spectra_rows)]
        ]

        # Ensure there is at least one row for empty top_spectra
        if top_spectra.empty:
            subplot_titles.extend([""] * (num_spectra_rows * num_columns - len(subplot_titles) + 5))
            specs.extend([[{"type": "scatter"} for _ in range(4)] for _ in range(num_spectra_rows - 1)])

        fig = make_subplots(
            rows=2 + num_spectra_rows,
            cols=4,
            shared_xaxes=False,
            shared_yaxes=yaxis_toggle.value,
            vertical_spacing=0.3 / (2 + num_spectra_rows),
            horizontal_spacing=0.1,
            subplot_titles=subplot_titles,
            specs=specs
        )

        # Add fallback traces if top_spectra is empty
        if top_spectra.empty:
            fig.add_trace(
                go.Scatter(
                    x=[],
                    y=[],
                    mode='lines',
                    name="No Spectra Available",
                    line=dict(color='gray'),
                    showlegend=False
                ),
                row=3,
                col=1
            )

        # Add the summary traces to the spanning subplot
        fig.update_xaxes(range=[x_min, x_max], row=1, col=3)  # Set x-axis bounds for the summary graph
        for trace in summary_traces:
            fig.add_trace(trace, row=1, col=3)  # Add to row 1, col 3

        # Add EIC traces for each adduct/peak
        for idx, (lcmsrun_path, eic) in enumerate(eics.items()):
            for i, eic_row in eic.iterrows():
                rt_sort = np.argsort(eic_row['rt'])
                adduct = sta.get_adduct(eic_row['label'])
                color = adduct_color[adduct]
                
                # Determine row and column for the current trace
                row = 1 if idx < 2 else 2
                col = (idx % 2) + 1
                
                # Dynamic facet_name determination 
                if row == 1 and col == 1:
                    facet_name = "Sample"
                elif row == 1 and col == 2:
                    facet_name = "Blank"
                elif row == 2 and col == 1:
                    facet_name = "Sample (Log)"
                elif row == 2 and col == 2:
                    facet_name = "Blank (Log)"

                # Add line traces for raw intensity
                trace_index = len(fig.data)
                fig.add_trace(
                    go.Scatter(
                        x=eic_row['rt'][rt_sort],
                        y=eic_row['i'][rt_sort],
                        mode='lines',
                        name=f"{adduct} {facet_name}",  # Include facet_name in legend
                        line=dict(color=color),
                        showlegend=True
                    ),
                    row=row,
                    col=col
                )

                # Recalculate facet_name for log-transformed traces
                if row + 1 == 2 and col == 1:
                    facet_name = "Sample (Log)"
                elif row + 1 == 2 and col == 2:
                    facet_name = "Blank (Log)"

                # Add line traces for log-transformed intensity
                trace_index = len(fig.data)
                fig.add_trace(
                    go.Scatter(
                        x=eic_row['rt'][rt_sort],
                        y=np.log10(eic_row['i'][rt_sort].astype(float)),
                        mode='lines',
                        name=f"{adduct} {facet_name}",  # Include updated facet_name in legend
                        line=dict(color=color),
                        showlegend=True
                    ),
                    row=row + 1,  # Log traces go to the next row
                    col=col
                )

                # Add peak markers for each peak associated with this adduct
                if not rt_peaks.empty:
                    if facet_name == "Sample" or facet_name == "Sample (Log)":
                        adduct_peaks = rt_peaks[rt_peaks['adduct'] == adduct]
                        for _, peak_info in adduct_peaks.iterrows():
                            peak_rt = peak_info['rt_peak']
                            peak_index = peak_info['peak_index']
                            peak_intensity = peak_info['intensity']

                            # Add marker for raw intensity
                            fig.add_trace(
                                go.Scatter(
                                    x=[peak_rt],
                                    y=[peak_intensity],
                                    mode='markers',
                                    marker=dict(color=color, size=10),
                                    name=f"{adduct} RT {peak_index}",
                                    showlegend=False
                                ),
                                row=row,
                                col=col
                            )

                            # Add marker for log-transformed intensity
                            fig.add_trace(
                                go.Scatter(
                                    x=[peak_rt],
                                    y=[np.log10(peak_intensity)],
                                    mode='markers',
                                    marker=dict(color=color, size=10),
                                    name=f"{adduct} RT {peak_index}",
                                    showlegend=False
                                ),
                                row=row + 1,  # Log traces go to the next row
                                col=col
                            )

                # Add MS2 spectra markers
                if not top_spectra.empty:
                    if facet_name == "Sample" or facet_name == "Sample (Log)":
                        adduct_spectra = top_spectra[top_spectra['adduct'] == adduct]
                        # Remove adduct filtering to show all MS2 spectra
                        for _, spectrum_row in adduct_spectra.iterrows():
                            spectrum_adduct = spectrum_row['adduct']
                            spectrum_peak_index = spectrum_row['peak_index']
                            rounded_rt = round(spectrum_row['rt'], 2)
                            marker_color = adduct_color.get(spectrum_adduct, 'gray')

                            # Find closest point in the current EIC
                            sorted_rt = eic_row['rt'][rt_sort]
                            sorted_intensity = eic_row['i'][rt_sort]
                            
                            # Skip if no intensity data available
                            if len(sorted_rt) == 0 or len(sorted_intensity) == 0:
                                continue
                                
                            # Find the closest RT point in the EIC
                            closest_idx = np.argmin(np.abs(sorted_rt - spectrum_row['rt']))
                            
                            if closest_idx >= len(sorted_rt):
                                # If the index is out of bounds, skip this spectrum
                                print(f"Warning: RT {spectrum_row['rt']} is out of bounds for EIC RT range.")
                                continue
                                
                            raw_intensity = sorted_intensity[closest_idx]
                            log_intensity = np.log10(raw_intensity)
                            
                            # Display marker for all spectra on the raw intensity plot
                            fig.add_trace(
                                go.Scatter(
                                    x=[spectrum_row['rt']],
                                    y=[raw_intensity],
                                    mode='markers',
                                    marker=dict(color=marker_color, symbol='x', size=10),
                                    name=f"MS2: {spectrum_adduct} ({spectrum_peak_index}) @ {rounded_rt}",
                                    showlegend=False
                                ),
                                row=row,
                                col=col
                            )
                            
                            # Display marker for all spectra on the log-transformed plot
                            fig.add_trace(
                                go.Scatter(
                                    x=[spectrum_row['rt']],
                                    y=[log_intensity],
                                    mode='markers',
                                    marker=dict(color=marker_color, symbol='x', size=10),
                                    name=f"MS2: {spectrum_adduct} ({spectrum_peak_index}) @ {rounded_rt}",
                                    showlegend=False
                                ),
                                row=row + 1,  # Log traces go to the next row
                                col=col
                            )

        # Add traces for Spectra plots
        if not top_spectra.empty:
            top_spectra_sorted = top_spectra.sort_values(['adduct', 'peak_index'])

            mz_list = [lst[0] for lst in top_spectra_sorted['spectrum'] if isinstance(lst, (list, np.ndarray)) and len(lst) > 0]
            mz_list_flattened = np.concatenate([np.ravel(arr) if isinstance(arr, np.ndarray) else np.array([arr]) for arr in mz_list])
            lowest_mz = np.min(mz_list_flattened)*0.9
            highest_mz = np.max(mz_list_flattened)*1.1

            for i, spectrum_row in enumerate(top_spectra_sorted.iterrows()):
                mz_values = spectrum_row[1]['spectrum'][0]
                i_values = spectrum_row[1]['spectrum'][1]
                adduct = spectrum_row[1]['adduct']
                color = adduct_color[adduct]
                precursor_mz = spectrum_row[1]['precursor_mz']
                peak_index = spectrum_row[1]['peak_index']
                spectrum_title = f"{adduct} ({peak_index}) @ {round(spectrum_row[1]['rt'], 2)} mins"

                # Determine the row and column for this spectrum
                spectrum_row_idx = 3 + (i // num_columns)  # Start after EIC rows
                spectrum_col = (i % num_columns) + 1

                # Update the x-axis range for the current subplot
                fig.update_xaxes(
                    range=[lowest_mz, highest_mz],  # Set x-axis limits
                    row=spectrum_row_idx,
                    col=spectrum_col
                )

                # Add vertical lines for each point
                for mz, intensity in zip(mz_values, i_values):
                    fig.add_trace(
                        go.Scatter(
                            x=[mz, mz],
                            y=[0, intensity],
                            mode='lines',
                            line=dict(color=color),
                            showlegend=False
                        ),
                        row=spectrum_row_idx,
                        col=spectrum_col
                    )

                # Add markers for each point
                fig.add_trace(
                    go.Scatter(
                        x=mz_values,
                        y=i_values,
                        mode='markers',
                        marker=dict(color=color, size=6),
                        name=f"Spectrum {i+1}: {adduct}",
                        showlegend=False
                    ),
                    row=spectrum_row_idx,
                    col=spectrum_col
                )

                # Add a black circle at precursor_mz (y=0)
                fig.add_trace(
                    go.Scatter(
                        x=[precursor_mz],
                        y=[0],
                        mode='markers',
                        marker=dict(color='black', symbol='circle', size=20),
                        name=f"Precursor MZ: {precursor_mz}",
                        showlegend=False
                    ),
                    row=spectrum_row_idx,
                    col=spectrum_col
                )
                
                fig.layout.annotations[5 + i].text = spectrum_title

        # Update layout
        fig_title = (group_id.replace('_', '  |  '))
        fig.update_layout(
            hoverlabel=dict(
                font_size=11,  # Increase font size for better readability
                namelength=-1  # Show the full name without truncation
            ),
            title=dict(text=fig_title, font=dict(size=14), x=0.5, xanchor="center"),
            height=700 + 300 * num_spectra_rows,
            width=1500,
            plot_bgcolor="white",
            paper_bgcolor="white",
            legend=dict(
                orientation="h",
                xanchor="center",
                yanchor="top",
                x=0.5,
                y=-0.2
            )
        )

        # Add black borders and keep gridlines
        fig.update_xaxes(
            showline=True,  # Show axis line
            linewidth=1,  # Set line width
            linecolor="black",  # Set line color to black
            showgrid=True,  # Keep gridlines
            gridcolor="lightgray"  # Set gridline color
        )
        fig.update_yaxes(
            showline=True,  # Show axis line
            linewidth=1,  # Set line width
            linecolor="black",  # Set line color to black
            showgrid=True,  # Keep gridlines
            gridcolor="lightgray"  # Set gridline color
        )

        # Update the compound image based on group_run_number
        if group_run_number in runnum_to_structure_image_grid:
            compound_image_widget.value = base64.b64decode(runnum_to_structure_image_grid[group_run_number])
            compound_image_label.value = f"Structures in Run{group_run_number}"  # Update the label text
        else:
            compound_image_widget.value = b''  # Clear the image if not found
            compound_image_label.value = "No Structures Found"  # Update the label text
        
        # Create new checkboxes for this unique_id
        all_adducts_checkboxes = []
        for combo in adduct_peak_combinations:
            # Check if this combo is selected
            is_selected = False
            if unique_id in selected_good_adducts:
                combo_id = f"{combo['adduct']}||{combo['peak_index']}"
                is_selected = combo_id in selected_good_adducts[unique_id]
            
            checkbox = widgets.Checkbox(
                value=is_selected,
                description=combo['description'],
                disabled=False,
                layout=widgets.Layout(width='300px', margin='0 0 0 -75px')
            )
            all_adducts_checkboxes.append(checkbox)
        
        # Create ambiguous checkbox
        ambiguous_checkbox = widgets.Checkbox(
            value=(unique_id in ambiguous_adducts),
            description="Ambiguous",
            disabled=False,
            layout=widgets.Layout(width='300px', margin='0 0 0 -75px')
        )
        all_adducts_checkboxes.append(ambiguous_checkbox)
        
        # Create top_adducts checkboxes
        top_adducts_options = list(dict.fromkeys(combo['adduct'] for combo in adduct_peak_combinations))
        top_adducts_children = []
        
        # Pre-select first adduct if this is a new unique_id
        default_to_first = unique_id not in top_adducts and len(top_adducts_options) > 0
        
        for idx, adduct in enumerate(top_adducts_options):
            # Check if this adduct should be selected
            is_selected = False
            
            if unique_id in top_adducts:
                # Use existing selection
                is_selected = adduct in top_adducts[unique_id]
            elif idx == 0 and default_to_first:
                # This is the first checkbox for a new unique_id - select it by default
                is_selected = True
                # Also update the dictionary
                top_adducts[unique_id] = [adduct]
                
            checkbox = widgets.Checkbox(
                value=is_selected,
                description=adduct,
                layout=widgets.Layout(width='275px', margin='0 0 0 -75px')
            )
            top_adducts_children.append(checkbox)
        
        top_adducts_checkboxes = widgets.VBox(children=top_adducts_children)
        
        # Create and attach handlers that explicitly use the current unique_id
        on_all_adducts_change, on_top_adducts_change = create_checkbox_handlers(
            unique_id, all_adducts_checkboxes, adduct_peak_combinations, ambiguous_checkbox, top_adducts_checkboxes
        )
        
        for checkbox in all_adducts_checkboxes:
            checkbox.observe(on_all_adducts_change, names='value')
        
        for checkbox in top_adducts_checkboxes.children:
            checkbox.observe(on_top_adducts_change, names='value')
        
        # Create layout with current widgets
        layout = create_layout(all_adducts_checkboxes, top_adducts_checkboxes)
        
        # Update image
        group_run_number = data['group_run_number']
        if group_run_number in runnum_to_structure_image_grid:
            compound_image_widget.value = base64.b64decode(runnum_to_structure_image_grid[group_run_number])
            compound_image_label.value = f"Structures in Run{group_run_number}"
        else:
            compound_image_widget.value = b''
            compound_image_label.value = "No Structures Found"
        
        # Display everything in the main output
        with main_output:
            display(layout)
            display(fig)  # Display the plotly figure
        
        update_progress_text()
    
    # Initial display
    update_display()
    
    # Build the main UI
    ui = widgets.VBox([
        main_output,
        output_container
    ])
    
    # Show the UI
    display(ui)

In [100]:
if config["selected_data_from_cache"] is False:
    processed_data = sta.process_data_for_plotting(eics_full, top_spectra_full, group_names_full, rt_peaks_full, config)
    selected_adducts_dict = {}
    ambiguous_adducts_dict = {}
    top_adducts_dict = {}
    create_interactive_plots(processed_data, mols_images, selected_adducts_dict, ambiguous_adducts_dict, top_adducts_dict)
    # Run next cell after manual selection of adducts

elif config["selected_data_from_cache"] is True:
    print("Not initiating GUI for adduct selection, loading selected adducts from cache below.")

In [102]:
selected_adducts_dict

{'sorgoleone;;/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run375.h5': ['[M+H]+||peak1',
  '[M+NH4]+||peak1'],
 'sorgoleone;;/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE205060norm-200uM-S1_Run376.h5': ['[M+H]+||peak1',
  '[M+NH4]+||peak1'],
 'sorgoleone;;/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_NEG_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run377.h5': ['[M-H]-||peak1'],
 'sorgoleone;;/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241

# Generate static summary reports for each compound after selections are made

In [98]:
if config["selected_data_from_cache"] is False:
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="selected", \
                    data=(selected_adducts_dict, ambiguous_adducts_dict, top_adducts_dict, processed_data))
    sta.generate_static_summary_plots(processed_data, selected_adducts_dict, top_adducts_dict, config) # Save summary plots of selected compounds+adducts after completing GUI

elif config["selected_data_from_cache"] is True:
    selected_adducts_dict, ambiguous_adducts_dict, top_adducts_dict, processed_data = sta.handle_data(mode="load", config=config, file_suffix="selected")

Saving data to: /global/homes/b/bkieft/metatlas_junkdrawer/example_data/schellermetasci/cache/20250506190611_ref_stds_selected.pkl


 Writing summary plots for selected compounds:   0%|          | 0/8 [00:00<?, ? compound group/s]

2025-05-06 21:09:41 CRITICAL Uncaught exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_805171/141666826.py", line 4, in <module>
    sta.generate_static_summary_plots(processed_data, selected_adducts_dict, top_adducts_dict, config) # Save summary plots of selected compounds+adducts after completing GUI
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/metatlas.ety4nerKCN/notebooks/standards_library/standard_annotation.py", line 3146, in generate_static_summary_plots
    fig.write_image(
  File "/usr/local/lib/python3.11/site-packages/plotly/basedatatypes.py", line 3827, in write_image
    return pio.write_image(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/plotly/io/_kale

SystemExit: 128

# Filter RT Peak, EICs, and Top Spectra by the selected compounds+adducts

In [58]:
if config["filtered_data_from_cache"] is False:
    eics_filtered, rt_peaks_filtered, top_spectra_filtered = sta.filter_by_selected(eics_full, rt_peaks_full, top_spectra_full, selected_adducts_dict, top_adducts_dict) 
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="filtered", \
                    data=(eics_filtered, top_spectra_filtered, rt_peaks_filtered))

elif config["filtered_data_from_cache"] is True:
    eics_filtered, top_spectra_filtered, rt_peaks_filtered = sta.handle_data(mode="load", config=config, file_suffix="filtered")
    print(f"\nTotal unique compounds selected: {eics_filtered['compound_name'].nunique()}")
    print(f"Total unique compound+adduct entries selected: {eics_filtered['label'].nunique()}\n")
    print(f"Total EICs selected: {eics_filtered.shape[0]}")
    print(f"Total RT peaks selected: {rt_peaks_filtered.shape[0]}")
    print(f"Total MS2 spectra selected: {top_spectra_filtered.shape[0]}")


Total unique compounds selected: 1
Total unique compound+adduct entries selected: 6

Total EICs selected: 6
Total RT peaks selected: 2
Total MS2 spectra selected: 2
Saving data to: /global/homes/b/bkieft/metatlas_junkdrawer/example_data/schellermetasci/cache/20250506190611_ref_stds_filtered.pkl


# Identify compounds not in the metatlas database Compounds table and store if necessary

In [59]:
if config['metatlas_db_data_from_cache'] is False:
    rt_peaks_filtered_all_selected, rt_peaks_filtered_top_selected = sta.format_and_select_top_adducts(rt_peaks_filtered, top_adducts_dict) # Export "all" for MSMS refs and "top" for EMA atlases
    in_db, notin_db = sta.search_for_matches_in_metatlas_db(rt_peaks_filtered_all_selected, check_by_flat=True) # Check if selected compounds are in metatlas DB
    if len(notin_db) > 0 and config['direct_store_to_compounds_table'] is True: # Store selected compounds+adducts in metatlas db
        sta.store_in_metatlas_db(notin_db)
    sta.check_db_deposit(rt_peaks_filtered_top_selected)
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="metatlas_db", \
                    data=(in_db, notin_db, rt_peaks_filtered_all_selected, rt_peaks_filtered_top_selected))

elif config['metatlas_db_data_from_cache'] is True:
    in_db, notin_db, rt_peaks_filtered_all_selected, rt_peaks_filtered_top_selected = sta.handle_data(mode="load", config=config, file_suffix="metatlas_db")
    print(f"\n'All' peaks dataset (for MSMS refs): {rt_peaks_filtered_all_selected.shape[0]} total compound peaks.")
    print(f"'Top' peaks dataset (for EMA atlases): {rt_peaks_filtered_top_selected.shape[0]} best compound peaks.\n")


Working on dataset: top
	Checking for differing RTs between CEs and polarities, which are unexpected...
		Group ('HILICZ', 'sorgoleone'): All RT values for ['102040norm'] and ['POS'] are within 0.05 mins of each other (0.0).

	Grouping by monoisotopic_mass and identify isomers in the datasets...
		No isomers found in top data.

	Selecting best collision energy row by intensity for the top adduct(s) per compound...
		Selected 1 row and removed 0 row(s) for ('HILICZ', 'POS', 'sorgoleone', '[M+H]+').
		Selected 1 row and removed 0 row(s) for ('HILICZ', 'POS', 'sorgoleone', '[M+NH4]+').

Working on dataset: all
	Checking for differing RTs between CEs and polarities, which are unexpected...
		Group ('HILICZ', 'sorgoleone'): All RT values for ['102040norm'] and ['POS'] are within 0.05 mins of each other (0.0).

	Grouping by monoisotopic_mass and identify isomers in the datasets...
		No isomers found in all data.

'All' peaks dataset (for MSMS refs): 2 total compound peaks.
'Top' peaks datas

 Searching for matches in metatlas db:   0%|          | 0/2 [00:00<?, ? compound/s]


Summary of compounds already in the metatlas database:



,query_label,query_matching_criterion,query_to_db,db_match
0,sorgoleone,inchi_key,FGWRUVXUQWGLOX-AFJQJTPPSA-N,[FGWRUVXUQWGLOX-AFJQJTPPSA-N]



All compounds are already in the metatlas database.

All new entries found in the database.

Saving data to: /global/homes/b/bkieft/metatlas_junkdrawer/example_data/schellermetasci/cache/20250506190611_ref_stds_metatlas_db.pkl


# Identify compounds+adducts not in atlases and set up new atlas creation

In [60]:
if config["ema_atlas_data_from_cache"] is False:
    ema_atlases_data = sta.get_ema_atlas_data(config["current_ema_atlases"])
    rt_peaks_filtered_top_selected_formatted = sta.convert_rt_peaks_to_atlas_format(rt_peaks_filtered_top_selected)
    matches_to_atlases, nonmatches_to_atlases = sta.search_for_matches_in_atlases(rt_peaks_filtered_top_selected_formatted, ema_atlases_data)

    if config["rt_correction_data_from_cache"] is False:
        print("Setting up RT correction for compounds not yet in atlases using baseline correction method:\n")
        baseline_qc, experimental_qc, baseline_to_experimental_qc = sta.get_qc_experimental_atlas(nonmatches_to_atlases, config["current_qc_atlases"], include_istds=True)
        baseline_correction_inputs = sta.create_baseline_correction_input(nonmatches_to_atlases, baseline_to_experimental_qc)
        baseline_correction_outputs = sta.rt_correction_from_baseline(baseline_correction_inputs, config)
        sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="rt_correction", \
                        data=(baseline_to_experimental_qc, baseline_correction_outputs))

    elif config["rt_correction_data_from_cache"] is True:
        baseline_to_experimental_qc, baseline_correction_outputs = sta.handle_data(mode="load",config=config, file_suffix="rt_correction")

    nonmatches_to_atlases_rt_corrected = sta.substitute_corrected_rt_values(nonmatches_to_atlases, baseline_correction_outputs)
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="ema_atlases", \
                    data=(nonmatches_to_atlases_rt_corrected, ema_atlases_data, baseline_to_experimental_qc, baseline_correction_outputs))

elif config["ema_atlas_data_from_cache"] is True:
    nonmatches_to_atlases_rt_corrected, ema_atlases_data, baseline_to_experimental_qc, baseline_correction_outputs = sta.handle_data(mode="load", config=config, file_suffix="ema_atlases")
    print(f"Total compounds to add to EMA atlases per chromatography: {nonmatches_to_atlases_rt_corrected['chromatography'].nunique()}")

 Searching in HILICZ positive atlas:   0%|          | 0/2 [00:00<?, ? compound/s]

 Searching in HILICZ negative atlas: 0 compound [00:00, ? compound/s]

 Searching in C18 positive atlas: 0 compound [00:00, ? compound/s]

 Searching in C18 negative atlas: 0 compound [00:00, ? compound/s]


None of the compounds+adducts searched were found in the atlases.

There are 2 compounds+adducts are not yet in any atlases. View with 'nonmatches_to_atlases'.

Setting up RT correction for compounds not yet in atlases using baseline correction method:

	Getting all QC files for project /global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558

	Retrieving baseline HILICZ QC atlas: /global/homes/b/bkieft/metatlas-data/HILIC/HILIC_QCv7_positive.tsv




...

Calculating RT correction model:   0%|          | 0/1 [00:00<?, ? chromatography/s]

	HILICZ RT correction results:


,label,adduct,polarity,rt_peak_baseline,rt_peak_experimental,rt_peak_corrected,rt_min_corrected,rt_max_corrected,rt_diff_experimental_vs_corrected
0,sorgoleone,[M+H]+,positive,NaN,0.801184,0.645454,0.145454,1.145454,0.15573
1,sorgoleone,[M+NH4]+,positive,NaN,0.801184,0.645454,0.145454,1.145454,0.15573
2,ABMBA (unlabeled),NaN,QC,1.093806,1.21834,1.096205,0.596205,1.596205,0.122135
3,N-acetyl-glucosamine (U - 13C),NaN,QC,6.707815,6.418233,6.580829,6.080829,7.080829,-0.162596
4,adenine (U - 15N),NaN,QC,2.677602,2.834389,2.827321,2.327321,3.327321,0.007068
5,"alanine (U - 13C, 15N)",NaN,QC,13.405091,13.240681,13.400469,12.900469,13.900469,-0.159788
6,"arginine (U - 13C, 15N)",NaN,QC,16.939915,17.013107,16.987934,16.487934,17.487934,0.025173
7,"asparagine (U - 13C, 15N)",NaN,QC,14.368089,14.252848,14.375830,13.875830,14.875830,-0.122982
8,"aspartic acid (U - 13C, 15N)",NaN,QC,16.130360,16.043652,16.078480,15.578480,16.578480,-0.034828
9,"cystine (U - 13C, 15N)",NaN,QC,16.904308,16.930342,16.910628,16.410628,17.410628,0.019714


Saving data to: /global/homes/b/bkieft/metatlas_junkdrawer/example_data/schellermetasci/cache/20250506190611_ref_stds_rt_correction.pkl
Formatted 2 RT-corrected compounds for insertion into HILICZ atlases.
Saving data to: /global/homes/b/bkieft/metatlas_junkdrawer/example_data/schellermetasci/cache/20250506190611_ref_stds_ema_atlases.pkl


/tmp/metatlas.ety4nerKCN/notebooks/standards_library/standard_annotation.py:1976: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.

/tmp/metatlas.ety4nerKCN/notebooks/standards_library/standard_annotation.py:1977: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.

/tmp/metatlas.ety4nerKCN/notebooks/standards_library/standard_annotation.py:1978: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the 

# Create new EMA atlas with top selected reference standards added

In [69]:
nonmatches_to_atlases_rt_corrected

,standard_lcmsrun,chromatography,compound_name,adduct,polarity,rt_peak,mz,smiles,peak_index,inchi,inchi_key,neutralized_inchi,neutralized_inchi_key,permanent_charge,formula,mono_isotopic_molecular_weight,collision_energy,label,rt_min,rt_max,mz_tolerance,mz_tolerance_units,in_metatlas
0,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run375.h5,HILICZ,sorgoleone,[M+H]+,positive,0.645454,359.221685,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",FGWRUVXUQWGLOX-AFJQJTPPSA-N,0,C22H30O4,358.214409,102040norm,sorgoleone,0.145454,1.145454,5,ppm,True
1,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run375.h5,HILICZ,sorgoleone,[M+NH4]+,positive,0.645454,376.248232,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",FGWRUVXUQWGLOX-AFJQJTPPSA-N,0,C22H30O4,358.214409,102040norm,sorgoleone,0.145454,1.145454,5,ppm,True


In [72]:
if config['save_new_ema_atlases'] is True:
    ema_atlas_ids, ema_atlas_names = sta.update_and_save_ema_atlases(nonmatches_to_atlases_rt_corrected, ema_atlases_data['HILICZ'], config, timestamp)
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="new_atlas_ids", \
                    data=(ema_atlas_ids, ema_atlas_names))
    ema_atlas_ids, ema_atlas_names = sta.handle_data(mode="load", config=config, file_suffix="new_atlas_ids")

    if config['direct_deposit_new_emas'] is True:
        print("New EMA atlases have been saved to disk and deposited in the metatlas database:")
        display(pd.DataFrame.from_dict(ema_atlas_ids))
    print(f"\nNew EMA atlas locations:")
    display(pd.DataFrame.from_dict(ema_atlas_names))

elif config['save_new_ema_atlases'] is False:
    print("No new EMA atlases saved to disk, as 'save_new_ema_atlases' is set to False in the config file.")
    print("Here is the new atlas data in memory:")
    display(nonmatches_to_atlases_rt_corrected)

2025-05-06 20:39:40 CRITICAL Uncaught exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_805171/3518290779.py", line 2, in <module>
    ema_atlas_ids, ema_atlas_names = sta.update_and_save_ema_atlases(nonmatches_to_atlases_rt_corrected, {ema_atlases_data['HILICZ']}, config, timestamp)
                                                                                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: unhashable type: 'dict'


SystemExit: 128

# Identify compounds not in MSMS refs and set up new MSMS refs creation

In [11]:
if config["msms_refs_data_from_cache"] is False:
    msms_refs = sta.get_msms_refs(msms_refs_path=config["current_msms_refs_path"])
    rt_peaks_filtered_all_selected_formatted = sta.format_for_msms_refs(rt_peaks_filtered_all_selected, top_spectra_filtered, msms_refs, config)
    in_msms_refs, notin_msms_refs = sta.search_for_matches_in_msms_refs(rt_peaks_filtered_all_selected_formatted, msms_refs, check_by_flat=True)
    sta.handle_data(mode="save", config=config, timestamp=timestamp, file_suffix="msms_refs", \
                    data=(msms_refs, notin_msms_refs, rt_peaks_filtered_all_selected_formatted))
    
elif config["msms_refs_data_from_cache"] is True:
    msms_refs, notin_msms_refs, rt_peaks_filtered_all_selected_formatted = sta.handle_data(mode="load", config=config, file_suffix="msms_refs")

Loaded MSMS refs with 216409 rows and 17 columns.


 Searching for matches in MSMS refs:   0%|          | 0/12 [00:00<?, ? compound/s]


12 compounds+adducts are not yet in MSMS refs. Check notin_msms_refs to view.

Saving data to: /global/homes/b/bkieft/metatlas/notebooks/standards_library/example_data/schellermetasci/cache/20250506103111_ref_stds_msms_refs.pkl


# Create new MSMS refs table with all selected reference standards added

In [12]:
if config['save_new_msms_refs'] is True:
    sta.update_and_save_msms_refs(msms_refs, notin_msms_refs, config, timestamp)

elif config['save_new_msms_refs'] is False:
    print("No new MSMS refs saved to disk, as 'save_new_msms_refs' is set to False in the config file.")
    print("Here is the new MSMS refs data in memory:")
    display(notin_msms_refs)

Existing MSMS refs went from 216409 to 216421 compounds.
	New MSMS refs: /global/homes/b/bkieft/metatlas/notebooks/standards_library/example_data/schellermetasci/updated_MSMS_refs/msms_refs_20250506103111.tab
